User info I extracted:  
(data dictionary: https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet)
* 'id'
* 'name'
* 'screen_name'
* 'location'
* 'created_at'
* 'followers_count'
* 'friends_count'      (
* 'statuses_count'     (The total number of tweets that the account has posted.)
* 'favourites_count'   (The total number of tweets that the account has liked.)


# Set up your PostgreSQL server
1.    Download and install Postgres.app from https://postgresapp.com/
2.   Open Postgres.app, toggle "start".
3.   Execute the code below to extract the records. Ensure that the files "corona-out-3" and "corona-out-3" are placed in the same working directory.

In [1]:
import psycopg2
import getpass

In [2]:
# prompting username input from the user
username = getpass.getuser()

In [3]:
# printing the username used
username

'varshiniyanamandra'

In [4]:
# Connect to the default "postgres" database to perform operations on databases
conn = psycopg2.connect(
    dbname="postgres",
    user=username,  # Only replace with your system user name
    password="",
    host="localhost",
    port="5432")

# Open a cursor to perform database operations
cur = conn.cursor()

# Check if the "twitter" database exists
cur.execute("SELECT 1 FROM pg_database WHERE datname = 'twitter';")
exists = cur.fetchone()

# Close the cursor and connection
cur.close()
conn.close()

if exists:
    # Create a new connection with autocommit enabled
    conn = psycopg2.connect(
        dbname="postgres",
        user=username,  # Only replace with your system user name
        password="",
        host="localhost",
        port="5432")
    conn.autocommit = True

    # Open a cursor to perform database operations
    cur = conn.cursor()

    # Drop and recreate the "twitter" database
    cur.execute("DROP DATABASE twitter;")
    cur.execute("CREATE DATABASE twitter;")

    # Close the cursor and connection
    cur.close()
    conn.close()
else:
    # Create a new connection with autocommit enabled
    conn = psycopg2.connect(
        dbname="postgres",
        user=username,  # Only replace with your system user name
        password="",
        host="localhost",
        port="5432")
    conn.autocommit = True

    # Open a cursor to perform database operations
    cur = conn.cursor()

    # create a new database named 'twitter'
    cur.execute("CREATE DATABASE twitter;")

    # Close the cursor and connection
    cur.close()
    conn.close()

# Connect to the newly created "twitter" database
conn = psycopg2.connect(
    dbname="twitter",
    user=username,  # Only replace with your system user name
    password="",
    host="localhost",
    port="5432")

# Open a cursor to perform further database operations
cur = conn.cursor()

print("PostgreSQL server information")
print(conn.get_dsn_parameters(), "\n")


PostgreSQL server information
{'user': 'varshiniyanamandra', 'passfile': '/Users/varshiniyanamandra/.pgpass', 'dbname': 'twitter', 'host': 'localhost', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



In [5]:
# Create the User table if it doesn't already exist
cur.execute('''
    CREATE TABLE IF NOT EXISTS TwitterUser (
        id BIGINT PRIMARY KEY,
        name TEXT,
        screen_name TEXT,
        location TEXT,
        created_at TIMESTAMP,
        followers_count INTEGER,
        friends_count INTEGER,
        statuses_count INTEGER,
        favourites_count INTEGER
    );
''')
conn.commit()  

In [6]:
def view_users(conn):
        try:
            cur.execute("SELECT * FROM TwitterUser;")
            users = cur.fetchall()
            if len(users)==0:
                print("No data")
            print("Number of user IDs:",len(users))
        except Exception as e:
            print(f"An error occurred: {e}")
            conn.rollback()

# Use this function to view the data in the TwitterUser table
view_users(conn)

No data
Number of user IDs: 0


In [7]:
import psycopg2
import json

def insert_user(cur, user_info):
        try:
            # Check if the user already exists in the database
            cur.execute("SELECT id FROM TwitterUser WHERE id = %s;", (user_info["id"],))
            if cur.fetchone() is None:
                # If the user does not exist, insert their information into the TwitterUser table
                cur.execute("""
                    INSERT INTO TwitterUser (id, name, screen_name, location, created_at, followers_count, friends_count, statuses_count, favourites_count)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
                """, (user_info["id"], user_info["name"], user_info["screen_name"], user_info["location"], user_info["created_at"], user_info["followers_count"], user_info["friends_count"], user_info["statuses_count"], user_info["favourites_count"]))
                conn.commit()
        except Exception as e:
            print(f"An error occurred: {e}")
            conn.rollback()

def insert_records(file):
    with open(file, "r") as f1:
        for line in f1:
            try:
                tweet = json.loads(line)

                # Extract the user information from the tweet
                user_info = tweet["user"]
                # Insert the user information into the User table
                insert_user(cur, user_info)

                # Check if the tweet is a retweet
                if "retweeted_status" in tweet:
                    # Extract the original tweet's user information
                    original_user_info = tweet["retweeted_status"]["user"]
                    # Insert the original tweet's user information into the User table
                    insert_user(cur, original_user_info)

            except:
                # if there is an error loading the json of the tweet, skip
                continue


In [8]:
#insert_records("corona-out-2")

In [9]:
#view_users(conn)

In [10]:
# cur.execute("SELECT * FROM TwitterUser;")
# users=cur.fetchall()
# for people in users:
#     print(people)

In [11]:
insert_records("corona-out-3")

In [12]:
view_users(conn)

Number of user IDs: 88068


In [ ]:
# Create separate indices for id and screen_name
cur.execute("CREATE INDEX idx_user_id ON TwitterUser (id);")
cur.execute("CREATE INDEX idx_user_screen_name ON TwitterUser (screen_name);")

# Commit changes and close connection
conn.commit()

In [13]:
# # Transform the data into a DataFrame for improved visualization and readability. 

# import pandas as pd
# from sqlalchemy import create_engine

# # Create an SQLAlchemy engine using the psycopg2 connection string
# url = "postgresql+psycopg2://" + username + ":@localhost:5432/twitter"
# # engine = create_engine("postgresql+psycopg2://varshiniyanamandra:@localhost:5432/twitter")
# engine = create_engine(url)

# def get_twitter_users_dataframe(engine):
#     query = "SELECT * FROM TwitterUser;"
#     df = pd.read_sql_query(query, engine)
#     return df

# # Fetch the data and convert it into a DataFrame
# user_df = get_twitter_users_dataframe(engine)


In [14]:
# user_df.head(20)

In [15]:
cur.close()
conn.close()